In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

In [2]:
train = pd.read_csv('train.csv')
y = train.target
test = pd.read_csv('./test.csv')
train.drop(['ID', 'target'], axis=1, inplace=True)
test.drop(['ID'], axis=1, inplace=True)
train.fillna(-9999, inplace=True)
test.fillna(-9999, inplace=True)
cat_features_ids = np.where(train.apply(pd.Series.nunique) < 30000)[0].tolist()

In [3]:
y1 = np.ones_like(y)
y2 = np.zeros((test.shape[0],))
y_all = np.hstack([y1, y2])
all_ = pd.concat([train, test])
clf = CatBoostClassifier(learning_rate=0.1, iterations=100, random_seed=42, eval_metric='AUC', logging_level='Silent')
clf.fit(all_, y_all, cat_features=cat_features_ids)
prediction = clf.predict(train)
best_val = train[prediction == 0]
print('Number of train samples from test distribution:', best_val.shape[0])

Number of train samples from test distribution: 49142


In [4]:
clf = CatBoostClassifier(learning_rate=0.1, iterations=100, random_seed=42, eval_metric='AUC', logging_level='Silent')
clf.fit(train.loc[prediction != 0, :], y[prediction != 0], cat_features=cat_features_ids)
prediction_val = clf.predict_proba(best_val)
print('Validation score:', roc_auc_score(y[prediction == 0], prediction_val[:, 1]))

Validation score: 0.7470119528903851
